# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
# load json as string
json.load((open('data/world_bank_projects_less.json')));

FileNotFoundError: [Errno 2] No such file or directory: 'data/world_bank_projects_less.json'

In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

ValueError: Expected object or value

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
import pandas as pd
import json
import numpy as np
from pandas.io.json import json_normalize

In [2]:
# load data set as Pandas dataframe
df = pd.read_json('world_bank_projects.json')
df.head(2)

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


In [3]:
df.columns

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')

### Exercise 1: Find the 10 countries with most projects

In [4]:
df1 = df.countryname.value_counts()
answer1 = df1.head(10)
answer1

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Nepal                              12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64

The countries above have the highest number of projects but that does not mean they also have the most mmoney invested. I will now look for the top 10 countries with the greatest total amount of money and then compare the two lists.

In [5]:
# Find the 10 countries with most projects (as in greatest total $ amount)

# look for correct $ amount of project
df[['grantamt','totalamt','totalcommamt']].head()

,grantamt,totalamt,totalcommamt
0,0,130000000,130000000
1,4700000,0,4700000
2,0,6060000,6060000
3,1500000,0,1500000
4,0,13100000,13100000


In [6]:
df1 = df[['countryname','totalcommamt']]   # choosing 'totalcommamt'
df1 = df1.groupby('countryname').sum()     # find sum for each country
answer2 = df1.sort_values(by='totalcommamt', ascending=False).head(10)    # sort
answer2

,totalcommamt
countryname,
Republic of India,2603700000
Federative Republic of Brazil,2326200000
Republic of Indonesia,2094740000
People's Republic of China,1966810000
Socialist Republic of Vietnam,1950640000
People's Republic of Bangladesh,1632240000
Republic of Poland,1307800000
Republic of Turkey,1304640000
Federal Democratic Republic of Ethiopia,1245000000


I will compare answers by determining if any of the countries with the most projects (answer1) are not in the list of those with the greatest total amounts (answer2). Such countries then must have only 'low_money_projects'.

In [7]:
# comparing two answers:
low_money_projects = ~answer1.index.isin(answer2.index)

In [8]:
answer1[low_money_projects].index

Index(['Republic of Yemen', 'Kingdom of Morocco', 'Nepal', 'Africa',
       'Republic of Mozambique'],
      dtype='object')

Yemen, Nepal, Morocco, Mozambique, and a generic 'Africa' have some of the highest number of projects but do not appear in the list of the top 10 highest grossing countries.

Similarly we can think of countries with high_money_projects if they show up in answer2 (high money)  but they are not in answer1 (thus having "few" projects).

In [9]:
high_money_projects = ~answer2.index.isin(answer1.index)

In [10]:
answer2[high_money_projects].index

Index(['Federative Republic of Brazil', 'Republic of Poland',
       'Republic of Turkey', 'Federal Democratic Republic of Ethiopia',
       'Federal Republic of Nigeria'],
      dtype='object', name='countryname')

Brazil, Poland, Turkey, Ethiopia, and Nigeria seem to have few but big projects. 
To further see this we list the 5 biggest projects, by 'totalcommamt' dollar amount.

In [11]:
df[['countryname','totalcommamt']].sort_values(by='totalcommamt',ascending=False).head(5)
#print(maxamt)
#df[df.totalcommamt == maxamt]

,countryname,totalcommamt
129,Republic of Poland,1307800000
147,Republic of Turkey,800000000
377,Republic of Indonesia,650000000
463,Federal Democratic Republic of Ethiopia,600000000
49,Republic of Colombia,600000000


We can see that Poland and Turkey have the two biggest projects, by dollar amounts, in particular Poland has only one massive project worth north of $1.3 billions.

### Exercise 2: Find the top 10 major project themes (using column 'mjtheme_namecode')

In [12]:
# Study column 'mjtheme_namecode
df2bis = df.mjtheme_namecode
df2bis[0]

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]

In [13]:
# load json as string
jsonstring = json.load((open('world_bank_projects.json')));

In [14]:
df2 = json_normalize(jsonstring, 'mjtheme_namecode')     # Normalize JSON string
df2 = df2[df2.name != '']                                # Remove empty name values
df2.groupby('name').size().sort_values(ascending=False).head(10)       # Find top 10 themes

name
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
dtype: int64

### Exercise 3: In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [15]:
# load json as string
jsonstring = json.load((open('world_bank_projects.json')));

In [17]:
df2 = json_normalize(jsonstring, 'mjtheme_namecode')
print('Dataframe has '+str(len(df2))+' rows')
# Next we replace '' values with NaN, in order to use the fillna method.
# Before using fillna, we sort values by code, so that the code of rows around an NaN value are similar.
df2 = df2.replace('',np.NaN)
df2 = df2.sort_values(by='code').fillna(method='ffill')

Dataframe has 1499 rows


In [18]:
# Checking that dataframe has been filled in. It should not have any null-objects
print(df2.info())
# There also should not be any '' value in 'name' column. Next line should return an empty dataframe.
#df2[df2.name == '']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1499 entries, 458 to 1102
Data columns (total 2 columns):
code    1499 non-null object
name    1499 non-null object
dtypes: object(2)
memory usage: 35.1+ KB
None


In [19]:
# An alternative way to check that dataframe has been filled in correctly!
df2['codename'] = df2.code+df2.name
len(df2.codename.unique()) == len(df2.code.unique())

True